In [2]:
import pandas as pd

## Impact of M and w/ (w/o) selection

In [5]:
def sample_maxGen(df, maxGen=20):
    df_all = None
    # print('OID = ', set(df['OID']))
    for oid in range(0, max(df['OID'])):
        if oid not in df['OID']:
            print('skip: oid = ', oid)
            continue
        df1 = df[df.OID == oid]
        if len(set(df1.pairConsistent.values)) == 2:
            dff = df1[df1.pairConsistent == False]
            if len(dff) >= maxGen:
                df1 = dff.iloc[:maxGen, :]
            else:
                dft = df1[df1.pairConsistent == True].iloc[:maxGen - len(dff), :]
                df1 = pd.concat([dff, dft], axis=0)
        else:
            df1 = df[df.OID == oid].iloc[:min(len(df1), maxGen), :]

        if df_all is None:
            df_all = df1
        else:
            df_all = pd.concat([df_all, df1], axis=0)
            # print(len(df_all))
        # print()
    return df_all

def setup_groundtruth(cr='NeuralCoref', gt_method='Textattack'):  # Stats
    # Because Checklist has all 200 sentences recorded, so we use it as ground truth.
    df = pd.read_csv('./{}/{}.tsv'.format(cr, gt_method), delimiter='\t')
    assert len(set(df['OID'])) == 1000

    Groundtruth_N = set(df[df['oriConsistent'] == False]['OID'])
    # print(len(Groundtruth_N))
    return Groundtruth_N

### NeuralCoref | M = 10, 20, 30, 50 | Select = True 

In [7]:
# rq4

Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='NeuralCoref')
N = len(Groundtruth_N)
P = Num_Src - N

select = True

for method in ('SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 'Crest', ): 
    print(f'\nMethod = {method}')
    df = pd.read_csv('./NeuralCoref/{}.tsv'.format(method), delimiter='\t')
    
    for m in [10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        if method == 'Crest' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 2)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')


Method = SIT
M: 10 | Select: True | Gen: 5518 | Issues: 1594 | Hit: 230 / 465 (49.46) | TP: 1102 | FP: 492 | Prec: 0.69 | CorefKeep: 3523 / 5518 =  0.64
M: 20 | Select: True | Gen: 10468 | Issues: 2338 | Hit: 230 / 465 (49.46) | TP: 1652 | FP: 686 | Prec: 0.71 | CorefKeep: 7000 / 10468 =  0.67
M: 30 | Select: True | Gen: 14602 | Issues: 2806 | Hit: 230 / 465 (49.46) | TP: 1999 | FP: 807 | Prec: 0.71 | CorefKeep: 10048 / 14602 =  0.69
M: 50 | Select: True | Gen: 21097 | Issues: 3194 | Hit: 230 / 465 (49.46) | TP: 2328 | FP: 866 | Prec: 0.73 | CorefKeep: 14957 / 21097 =  0.71

Method = PatInv
M: 10 | Select: True | Gen: 4773 | Issues: 1178 | Hit: 167 / 465 (35.91) | TP: 840 | FP: 338 | Prec: 0.71 | CorefKeep: 3710 / 4773 =  0.78
M: 20 | Select: True | Gen: 9425 | Issues: 1880 | Hit: 167 / 465 (35.91) | TP: 1397 | FP: 483 | Prec: 0.74 | CorefKeep: 7478 / 9425 =  0.79
M: 30 | Select: True | Gen: 13945 | Issues: 2372 | Hit: 167 / 465 (35.91) | TP: 1777 | FP: 595 | Prec: 0.75 | CorefKeep: 1

### NeuralCoref | M = 10, 20, 30, 50 | Select = False 

In [9]:
Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='NeuralCoref')
N = len(Groundtruth_N)
P = Num_Src - N

select = False

for method in ('SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 'Crest',  ): #  , 'SIT', 
    print(f'\nMethod = {method}')
    df = pd.read_csv('./NeuralCoref/{}.tsv'.format(method), delimiter='\t')
    
    for m in [10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        if method == 'Crest' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 2)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')


Method = SIT
M: 10 | Select: False | Gen: 5518 | Issues: 1594 | Hit: 230 / 465 (49.46) | TP: 1102 | FP: 492 | Prec: 0.69 | CorefKeep: 3523 / 5518 =  0.64
M: 20 | Select: False | Gen: 10468 | Issues: 2338 | Hit: 230 / 465 (49.46) | TP: 1652 | FP: 686 | Prec: 0.71 | CorefKeep: 7000 / 10468 =  0.67
M: 30 | Select: False | Gen: 14602 | Issues: 2806 | Hit: 230 / 465 (49.46) | TP: 1999 | FP: 807 | Prec: 0.71 | CorefKeep: 10048 / 14602 =  0.69
M: 50 | Select: False | Gen: 21097 | Issues: 3194 | Hit: 230 / 465 (49.46) | TP: 2328 | FP: 866 | Prec: 0.73 | CorefKeep: 14957 / 21097 =  0.71

Method = PatInv
M: 10 | Select: False | Gen: 4773 | Issues: 1178 | Hit: 167 / 465 (35.91) | TP: 840 | FP: 338 | Prec: 0.71 | CorefKeep: 3710 / 4773 =  0.78
M: 20 | Select: False | Gen: 9425 | Issues: 1880 | Hit: 167 / 465 (35.91) | TP: 1397 | FP: 483 | Prec: 0.74 | CorefKeep: 7478 / 9425 =  0.79
M: 30 | Select: False | Gen: 13945 | Issues: 2372 | Hit: 167 / 465 (35.91) | TP: 1777 | FP: 595 | Prec: 0.75 | Coref

### StatsCR | M = 10, 20, 30, 50 | Select = True 

In [10]:
Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='Stats')
N = len(Groundtruth_N)
P = Num_Src - N

select = True

for method in ('SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 'Crest',  ): #  , 'SIT', 
    print(f'\nMethod = {method}')
    df = pd.read_csv('./Stats/{}.tsv'.format(method), delimiter='\t')

    for m in [10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        if method == 'Crest' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 2)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')


Method = SIT
M: 10 | Select: True | Gen: 5518 | Issues: 1544 | Hit: 143 / 365 (39.18) | TP: 997 | FP: 547 | Prec: 0.65 | CorefKeep: 3448 / 5518 =  0.62
M: 20 | Select: True | Gen: 10468 | Issues: 2174 | Hit: 143 / 365 (39.18) | TP: 1422 | FP: 752 | Prec: 0.65 | CorefKeep: 6897 / 10468 =  0.66
M: 30 | Select: True | Gen: 14602 | Issues: 2498 | Hit: 143 / 365 (39.18) | TP: 1626 | FP: 872 | Prec: 0.65 | CorefKeep: 9945 / 14602 =  0.68
M: 50 | Select: True | Gen: 21097 | Issues: 2751 | Hit: 143 / 365 (39.18) | TP: 1795 | FP: 956 | Prec: 0.65 | CorefKeep: 14836 / 21097 =  0.7

Method = PatInv
M: 10 | Select: True | Gen: 4773 | Issues: 957 | Hit: 78 / 365 (21.37) | TP: 712 | FP: 245 | Prec: 0.74 | CorefKeep: 3689 / 4773 =  0.77
M: 20 | Select: True | Gen: 9425 | Issues: 1410 | Hit: 78 / 365 (21.37) | TP: 1059 | FP: 351 | Prec: 0.75 | CorefKeep: 7362 / 9425 =  0.78
M: 30 | Select: True | Gen: 13945 | Issues: 1690 | Hit: 78 / 365 (21.37) | TP: 1250 | FP: 440 | Prec: 0.74 | CorefKeep: 11036 / 

### StatsCR | M = 10, 20, 30, 50 | Select = False 

In [11]:
Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='Stats')
N = len(Groundtruth_N)
P = Num_Src - N

select = False

for method in ('SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 'Crest',  ): #  , 'SIT', 
    print(f'\nMethod = {method}')
    
    df = pd.read_csv('./Stats/{}.tsv'.format(method), delimiter='\t')
    
    for m in [10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        if method == 'Crest' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 2)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')


Method = SIT
M: 10 | Select: False | Gen: 5518 | Issues: 1544 | Hit: 143 / 365 (39.18) | TP: 997 | FP: 547 | Prec: 0.65 | CorefKeep: 3448 / 5518 =  0.62
M: 20 | Select: False | Gen: 10468 | Issues: 2174 | Hit: 143 / 365 (39.18) | TP: 1422 | FP: 752 | Prec: 0.65 | CorefKeep: 6897 / 10468 =  0.66
M: 30 | Select: False | Gen: 14602 | Issues: 2498 | Hit: 143 / 365 (39.18) | TP: 1626 | FP: 872 | Prec: 0.65 | CorefKeep: 9945 / 14602 =  0.68
M: 50 | Select: False | Gen: 21097 | Issues: 2751 | Hit: 143 / 365 (39.18) | TP: 1795 | FP: 956 | Prec: 0.65 | CorefKeep: 14836 / 21097 =  0.7

Method = PatInv
M: 10 | Select: False | Gen: 4773 | Issues: 957 | Hit: 78 / 365 (21.37) | TP: 712 | FP: 245 | Prec: 0.74 | CorefKeep: 3689 / 4773 =  0.77
M: 20 | Select: False | Gen: 9425 | Issues: 1410 | Hit: 78 / 365 (21.37) | TP: 1059 | FP: 351 | Prec: 0.75 | CorefKeep: 7362 / 9425 =  0.78
M: 30 | Select: False | Gen: 13945 | Issues: 1690 | Hit: 78 / 365 (21.37) | TP: 1250 | FP: 440 | Prec: 0.74 | CorefKeep: 1